In [10]:
# Open file and parse contents
file = open('day11_inputs.txt')
content = [line.strip() for line in file]

row_width = len(content[0])
n_rows = len(content)
print(row_width, n_rows)

99 91


### Part 1

These general rules apply:
- L denotes an empty seat
- . denotes floor
- Neither seats or floor move
- 'adjacency' applies to the **8** spaces next to a given square - up/down/left/right/diagonal

These rules apply for changing the states of seats:
- If a seat is empty (L) and there are no occupied seats adjacent to it, the seat becomes occupied.
- If a seat is occupied (#) and four or more seats adjacent to it are also occupied, the seat becomes empty.
- Otherwise, the seat's state does not change
- These rules are applied to all seats simultaneously in a round of changes, i.e. a seat becoming occupied in round 1 won't be considered by the seat next to it in it's state calculations for round 1 - but it will be for round 2

In [2]:
import itertools
import copy

def adjacent_occupancy(seat_row, seat_col, content):
# Given a seat row/column, return n occupied seats adjacent to that seat    

    # Get locations of adjacent seats
    adjacent_indices = [-1,0,1]
    iter_adjacent_indices = itertools.product(adjacent_indices, adjacent_indices)
    seats_occupied = 0
    
    for adj in iter_adjacent_indices:
        # Unpack into row/col
        adj_row, adj_col = adj 
            
        # Get relative co-ordinates
        adj_row_ind = adj_row + seat_row
        adj_col_ind = adj_col + seat_col        
        
        # Ignore 0/0
        if adj_row==0 and adj_col==0:
            continue
        # Check validity of index
        elif (adj_row_ind < 0) or (adj_row_ind > n_rows-1) or (adj_col_ind < 0) or (adj_col_ind > row_width-1):
            continue
        # Otherwise, check number of adjacent '#'s (occupied seats)
        elif content[adj_row_ind][adj_col_ind] == '#':
            seats_occupied += 1
        else:
            continue

    return seats_occupied
            
        
# Create iterables
range_rows = list(range(n_rows))
range_cols = list(range(row_width))
content_copy = copy.deepcopy(content)
content_new = copy.deepcopy(content)

# Iterate
for i in range(1000):
    
    for seat_row in range_rows:
        new_row = ''
        for seat_col in range_cols:
            symbol = content_new[seat_row][seat_col]

            if symbol == 'L' and adjacent_occupancy(seat_row, seat_col, content_new) == 0:
                new_row+='#'
            elif symbol == '#' and adjacent_occupancy(seat_row, seat_col, content_new) > 3:
                new_row+='L'
            else:
                new_row+=symbol
        
        content_copy[seat_row] = new_row
        
        
    if content_copy == content_new:
        total_occupied = 0
        for row in content_copy:
            total_occupied += row.count('#')
        print('No change on loop {}, total seats occupied {}'.format(i+1, 
                                                                     total_occupied))
        break
        
    # Overwrite ready for next iteration
    content_new = copy.deepcopy(content_copy)

No change on loop 76, total seats occupied 2354


### Part 2

New rules:
- People don't care about adjacent seats, just the **first seat** that they can **see** in each direction

In [11]:
def is_valid_index(row_ind, col_ind):
# Check whether a set of row/col indices are in bounds
    return (row_ind >= 0) and (row_ind <= n_rows-1) and (col_ind >= 0) and (col_ind <= row_width-1)

def visual_occupancy(seat_row, seat_col, content):
# Given a seat row/column, return n occupied seats adjacent to that seat    

    # Get locations of adjacent seats
    adjacent_indices = [-1,0,1]
    iter_adjacent_indices = itertools.product(adjacent_indices, adjacent_indices)
    seats_occupied = 0
    
    for adj in iter_adjacent_indices:
        # Unpack into row/col
        adj_row, adj_col = adj        
        
        # Ignore 0/0
        if adj_row==0 and adj_col==0:
            continue
        
        adj_row_ind = seat_row + adj_row
        adj_col_ind = seat_col + adj_col
        
        # Only start loop for invalid indices
        if is_valid_index(adj_row_ind, adj_col_ind):
            
            # Get 'visible' indices
            vis_row_ind, vis_col_ind = adj_row_ind, adj_col_ind
            while is_valid_index(vis_row_ind, vis_col_ind):
                
                if content[vis_row_ind][vis_col_ind] == '#':
                    seats_occupied += 1
                    break
                elif content[vis_row_ind][vis_col_ind] == 'L':
                    break
                else:
                    # If no seat spotted, update indices
                    if adj_col > 0:
                        vis_col_ind+=1
                    if adj_col < 0:
                        vis_col_ind-=1
                        
                    if adj_row > 0:
                        vis_row_ind+=1
                    if adj_row < 0:
                        vis_row_ind-=1

    return seats_occupied

In [15]:
# Create iterables
range_rows = list(range(n_rows))
range_cols = list(range(row_width))
content_copy_vis = copy.deepcopy(content)
content_new_vis = copy.deepcopy(content)

# Iterate
for i in range(1000):
    
    for seat_row in range_rows:
        new_row = ''
        for seat_col in range_cols:
            symbol = content_new_vis[seat_row][seat_col]

            if symbol == 'L' and visual_occupancy(seat_row, seat_col, content_new_vis) == 0:
                new_row+='#'
            elif symbol == '#' and visual_occupancy(seat_row, seat_col, content_new_vis) > 4:
                new_row+='L'
            else:
                new_row+=symbol
        
        content_copy_vis[seat_row] = new_row
        
        
    if content_copy_vis == content_new_vis:
        total_occupied_vis = 0
        for row in content_copy_vis:
            total_occupied_vis += row.count('#')
        print('No change on loop {}, total seats occupied {}'.format(i+1, 
                                                                     total_occupied_vis))
        break
        
    # Overwrite ready for next iteration
    content_new_vis = copy.deepcopy(content_copy_vis)

No change on loop 84, total seats occupied 2072
